# Run for setup

## Drive connection

In [25]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

!ls '/content/drive/My Drive/SonART'

Mounted at /content/drive
Audio_Matteo			ocean-cruise-liner-ship-32308.mp3
audio_prova.mp3			ProjectMPT_DEFINITIVO1.mp3
birds-chirping-75156.mp3	prova2_2.wav
city-street-downtown-52115.mp3	prova3_2.wav
Code				river-26984.mp3
Demo				saved_model
filterable_labels.json		sea-and-seagull-wave-5932.mp3
Fog_Horn.mp3			seagull-sound-effect-272695.mp3
Fog-Horn.wav			sea-waves-169411.mp3
generated_images		ship-engine-32705.mp3
labels_config.json		subgroups_label.json
mar-calmado-272997.mp3


## Import

In [3]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
!pip install torchlibrosa

In [6]:
import os
import sys
import numpy as np
import librosa
import torch
import datetime
from IPython.display import Audio, clear_output, display
import ipywidgets as widgets
from google.colab import files
import json

#images model
from diffusers import LCMScheduler, AutoPipelineForText2Image

#GUI
import re
import io
import contextlib

#Connectiobn with processing
import socket
import cv2
import base64
#import time

In [7]:
#recover audio model code
path = "/content/drive/MyDrive/SonART/Code"
if os.path.exists(path):
    print("The folder exists!")
else:
    print("Error: Folder not found. Check the path.")

sys.path.append("/content/drive/MyDrive/SonART/Code")
from panns_inference import AudioTagging, labels

The folder exists!


In [9]:
change_labels_path = "/content/drive/MyDrive/SonART/labels_config.json"

with open(change_labels_path, "r") as f:
    labels_mapping = json.load(f)

groups = labels_mapping["groups"]

In [10]:
filterable_labels_path = "/content/drive/MyDrive/SonART/filterable_labels.json"

with open(filterable_labels_path, "r") as f:
    filterable_mapping = json.load(f)

filterable_labels = filterable_mapping["filterable_labels"]

## Processing connection

In [24]:
# Server data
host = '6.tcp.eu.ngrok.io'
port = 12250

## Retrieve Audio model

In [13]:
#It is possible to indicate the weight and model paths, if they are present then they will simply be downloaded,
#otherwise they will be created and saved in the indicated paths. If the paths are not specified, then the default ones are used

checkpoint_path = '/content/drive/My Drive/SonART/saved_model/panns/Cnn14_mAP=0.431.pth'
checkpoint_model = '/content/drive/My Drive/SonART/saved_model/panns/panns_model.pth'

device = 'cuda' # 'cuda' | 'cpu'
at = AudioTagging(checkpoint_path=checkpoint_path, checkpoint_model=checkpoint_model, device=device)

Checkpoint path: /content/drive/My Drive/SonART/saved_model/panns/Cnn14_mAP=0.431.pth
✅ File already exists, no download needed.
✅ Model uploaded by: /content/drive/My Drive/SonART/saved_model/panns/panns_model.pth
GPU number: 1


## Download image model

In [14]:
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
adapter_id = "latent-consistency/lcm-lora-sdxl"

pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# load and fuse lcm lora
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

...

vae_1_0/diffusion_pytorch_model.fp16.saf(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


pytorch_lora_weights.safetensors:   0%|          | 0.00/394M [00:00<?, ?B/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


## Functions and classes needed for audio + image model

In [15]:
def best_tag(clipwise_output):
    """Visualization of audio tagging result.

    Args:
      clipwise_output: (classes_num,)
    """
    sorted_indexes = np.argsort(clipwise_output)[::-1]

    return sorted_indexes[0]

In [16]:
class ObservableList(list):
    def __init__(self, *args):
        super().__init__(*args)
        self.save_dir = None
        self.saved_images = {}

    def set_audio_path(self, audio_path, base_dir= "/content/images"):
        file_name = os.path.basename(audio_path)
        audio_name = os.path.splitext(file_name)[0]
        self.save_dir = os.path.join(base_dir, audio_name)
        os.makedirs(self.save_dir, exist_ok=True)

    def append(self, item, i, background, style, time_stamp):
        super().append(item + str(i))
        self.on_append(item, i, background, style, time_stamp)

    def on_append(self, item, i, background, style, time_stamp):
        prompt = item
        if background is not None:
            prompt += f" with {background} as background"
        if style is not None:
            prompt += f" in {style} style"

        image = pipe(prompt=prompt, num_inference_steps=5, guidance_scale=0).images[0]

        millis_stamp = int(time_stamp * 1000)
        filename = f"{self.save_dir}/{millis_stamp}.png"

        # Save image
        image.save(filename)
        self.saved_images[item + str(i)] = filename

    def get_saved_image(self, item):
        """Returns the path of the saved image associated with the item"""
        return self.saved_images.get(item, None)

In [17]:
def generate_and_play_audio_with_images(audio_path, tag_list):

  display(Audio(audio_path, autoplay=True))

  for i, element in enumerate(tag_list):
      image_path = tag_list.get_saved_image(element)
      image = cv2.imread(image_path)

      timestamp = os.path.splitext(os.path.basename(image_path))[0]

      # Image coding
      _, img_encoded = cv2.imencode('.jpg', image)
      img_base64 = base64.b64encode(img_encoded.tobytes()).decode('utf-8')

      try:
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client.connect((host, port))

        client.sendall((f"{timestamp}\n").encode('utf-8'))         # filename
        client.sendall((f"{img_base64}\n").encode('utf-8'))        # base64

        print(f"📤 Sent image with timestamp {timestamp}")
        client.close()

      except Exception as e:
          print(f"⚠️ Error connecting or sending: {e}")

In [18]:
def replace_if_trouble_label(input_string):

    for group, values in groups.items():
        if input_string in values:
            return group

    return input_string

In [19]:
def elaborate(audio_path, background, style, prob_value, excluded_labels):

    sr = 32000
    audio, sr = librosa.load(audio_path, sr=sr, mono=True)

    # Specifies the duration of the segments in seconds
    segment_duration = 2
    hop_duration = 0.1

    # Calculate the length of each segment in samples
    segment_length = int(segment_duration * sr)
    hop_length = int(hop_duration * sr)

    # Divide the waveform into segments with the specified hop
    segments = []
    for start in range(0, len(audio) - segment_length + 1, hop_length):
        segment = audio[start:start + segment_length]
        segments.append(segment)

    # Add last segment if not covered by loop (padding if necessary)
    if len(audio) % hop_length != 0:
        last_segment = audio[-segment_length:]
        segments.append(last_segment)

    # Transform segments into a numpy array
    segments = np.stack(segments)

    # Create time stamps
    time_stamps = np.arange(len(segments)) * hop_duration

    print(f"Numero di segmenti: {len(segments)}, Forma di ogni segmento: {segments[0].shape}")

    # Variable to track the previously printed word
    last_printed_word = None

    tag_list = ObservableList()
    tag_list.set_audio_path(audio_path)
    last_event = None

    for i, segment in enumerate(segments):
        segment = segment[None, :]  # (batch_size, segment_samples)

        (clipwise_output, embedding) = at.inference(segment)
        """clipwise_output: (batch_size, classes_num), embedding: (batch_size, embedding_size)"""

        # Filter excluded labels
        if  excluded_labels is not None:
            labels_str = [str(label) for label in labels]
            mask = np.isin(labels_str, excluded_labels)
            clipwise_output[0][mask] = 0

        index = best_tag(clipwise_output[0])
        predicted_class = np.array(labels)[index]

        # Change label to improve image generation
        predicted_class = replace_if_trouble_label(predicted_class)

        clipwise_output = clipwise_output[0]

        # Print if it is not equal to the last print and exceeds the threshold
        if predicted_class != last_printed_word:
            if clipwise_output[index] >= prob_value:
                print('{}: {:.3f}'.format(predicted_class,
                clipwise_output[index]))
                print(str(datetime.timedelta(seconds=time_stamps[i])))
                tag_list.append(predicted_class, i, background, style, time_stamps[i])
                last_printed_word = predicted_class
                last_event = i

    return tag_list


# GUI

In [26]:
# ──────────── GUI ────────────
text1 = widgets.Text(value='nature', placeholder='Es. snowy forest', description='🎨 Background:', layout=widgets.Layout(width='70%'))
text2 = widgets.Text(value='fantasy art', placeholder='Es. watercolor style', description='🖌️ Style:', layout=widgets.Layout(width='70%'))

checkboxes = [widgets.Checkbox(value=False, description=label) for label in filterable_labels]
rows = [widgets.HBox(checkboxes[i:i+4]) for i in range(0, len(checkboxes), 4)]
checkbox_title = widgets.HTML("<b>Seleziona le etichette da escludere:</b>")
checkbox_box = widgets.VBox([checkbox_title] + rows)

def get_excluded_labels():
    return [cb.description for cb in checkboxes if cb.value]

# Widget to display the uploaded file
file_info = widgets.HTML(value="<div style='padding:5px; color:gray;'>No files loaded</div>")
upload_button = widgets.Button(description='📤 Upload File', button_style='primary', layout=widgets.Layout(width='50%'))

# Widget for selecting processed audio
processed_audios = {}
audio_dropdown = widgets.Dropdown(
    options=[],
    description='Audio:',
    disabled=True,
    layout=widgets.Layout(width='70%')
)


play_button = widgets.Button(
    description='▶️ Play',
    button_style='success',
    disabled=True,
    layout=widgets.Layout(width='50%')
)

status_text = widgets.HTML(value="<div style='padding: 5px;'>Pronto per l'upload</div>")

prob_slider = widgets.FloatSlider(
    value=0.4,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Theshold:',
    continuous_update=False,
    readout_format='.2f',
    layout=widgets.Layout(width='70%')
)

filter_toggle = widgets.ToggleButton(
    value=True,
    description="Lebel excluded",
    button_style='warning',
    tooltip='Enable/Disable Excluded Labels Filter',
    layout=widgets.Layout(width='50%')
)

# Global variables
uploaded_file_path = None
processed_files = {}

def on_toggle_filter(change):
    global use_excluded_labels
    use_excluded_labels = change['new']
    if use_excluded_labels:
        filter_toggle.description = "Lebel excluded"
        filter_toggle.button_style = "warning"
    else:
        filter_toggle.description = "All Label included"
        filter_toggle.button_style = "success"

filter_toggle.observe(on_toggle_filter, names='value')
# ──────────── Loading function ────────────
def upload_and_process_handler(_):
    global uploaded_file_path, processed_files
    clear_output(wait=True)
    display(final_ui)
    try:
        with contextlib.redirect_stdout(io.StringIO()):
           uploaded = files.upload()

        if not uploaded:
            file_info.value = "<div style='color:red; padding:5px;'>No files selected</div>"
            return

        filename = next(iter(uploaded.keys()))
        content = uploaded[filename]

        uploaded_file_path = f"/content/{filename}"

        # Check if the file has a valid audio extension
        valid_extensions = [".mp3", ".wav", ".flac", ".ogg", ".m4a", ".aac"]
        if not any(filename.lower().endswith(ext) for ext in valid_extensions):
            file_info.value = f"<div style='color:red; padding:5px;'>Invalid file type. Only audio files are accepted</div>"
            return

        with open(uploaded_file_path, 'wb') as f:
            f.write(content)

        # Check if the name contains "(number)"
        if re.search(r"\s\(\d+\)", filename):
            os.remove(uploaded_file_path)
            file_info.value = "<div style='color:red; padding:5px;'>File already uploaded</div>"
            status_text.value = "<div style='color:red; padding:5px;'>Upload cancelled</div>"
            return

        file_info.value = f"<div style='color:green; padding:5px;'>File uploaded: <b>{filename}</b></div>"
        status_text.value = "<div style='color:green; padding:5px;'>File ready for processing</div>"


        # Run the processing
        background = text1.value
        style = text2.value
        prob = prob_slider.value
        excluded_labels = get_excluded_labels()
        excluded_labels = [str(label) for label in excluded_labels]

        tag_list = elaborate(uploaded_file_path, background, style, prob, excluded_labels)

        # Save the results
        processed_files[filename] = {
            'file_path': uploaded_file_path,
            'tag_list': tag_list
        }

        # Update dropdown
        audio_dropdown.options = list(processed_files.keys())
        audio_dropdown.disabled = False
        play_button.disabled = False

        status_text.value = f"<div style='color:green; padding:5px;'>Processing completed! {len(processed_files)} ready files</div>"


    except Exception as e:
        file_info.value = f"<div style='color:red; padding:5px;'>Errore: {str(e)}</div>"
        status_text.value = "<div style='color:red; padding:5px;'>Errore durante l'elaborazione</div>"

upload_button.on_click(upload_and_process_handler)

# ──────────── Playback function ────────────
def play_selected_audio(_):
    clear_output(wait=True)
    display(final_ui)
    selected_audio = audio_dropdown.value
    if not selected_audio or selected_audio not in processed_files:
        status_text.value = "<div style='color:red; padding:5px;'>No files selected!</div>"
        return

    file_data = processed_files[selected_audio]
    status_text.value = "<div style='color:blue; padding:5px;'>Playing...</div>"

    try:
        generate_and_play_audio_with_images(file_data['file_path'], file_data['tag_list'])
        status_text.value = "<div style='color:green; padding:5px;'>Playback completed!</div>"
    except Exception as e:
        status_text.value = f"<div style='color:red; padding:5px;'>Error while playing: {str(e)}</div>"

play_button.on_click(play_selected_audio)


#------------ file deletion -----------
def list_uploaded_files():
    files = [f for f in os.listdir('/content') if os.path.isfile(os.path.join('/content', f))]
    return files

file_checkboxes = []

def refresh_file_list():
    global file_checkboxes
    file_list = list_uploaded_files()
    file_checkboxes = [widgets.Checkbox(value=False, description=f, indent=False) for f in file_list]
    file_list_box.children = file_checkboxes

file_list_box = widgets.VBox()
refresh_button = widgets.Button(description='🔄 Update list', layout=widgets.Layout(width='40%'))
delete_button = widgets.Button(description='🗑️ Delete selected audio', button_style='danger', layout=widgets.Layout(width='50%'))
delete_status = widgets.HTML("")

def on_refresh_button_click(_):
    refresh_file_list()
    delete_status.value = "<div style='color:gray; padding:5px;'>Updated list</div>"

def on_delete_button_click(_):
    deleted = []
    for cb in file_checkboxes:
        if cb.value:
            filename = cb.description
            try:
                os.remove(os.path.join('/content', filename))
                deleted.append(filename)

                # Also remove from processed files
                if filename in processed_files:
                    del processed_files[filename]
                    # Also update the dropdown if needed
                    audio_dropdown.options = list(processed_files.keys())
                    if not processed_files:
                        audio_dropdown.disabled = True
                        play_button.disabled = True

            except Exception as e:
                delete_status.value = f"<div style='color:red; padding:5px;'>Error while deleting: {str(e)}</div>"
                return

    refresh_file_list()
    delete_status.value = f"<div style='color:green; padding:5px;'>Eliminated: {', '.join(deleted)}</div>" if deleted else "<div style='padding:5px;'>No files selected</div>"

refresh_button.on_click(on_refresh_button_click)
delete_button.on_click(on_delete_button_click)

delete_ui = widgets.VBox([
    widgets.HTML("<h4>Gestione file caricati in <code>/content</code>:</h4>"),
    file_list_box,
    widgets.HBox([refresh_button, delete_button]),
    delete_status
], layout=widgets.Layout(padding='8px', width='400px',border='1px solid lightgray'))

# ──────────── Layout  ────────────
final_ui = widgets.VBox([
    widgets.HTML("<h2 style='color:#4CAF50;'>🎼 SonART - Audio to Image Generator</h2>"),
    widgets.VBox([text1, text2, prob_slider, checkbox_box]),
    widgets.HTML("<b>📂 Upload and process an audio file:</b>"),
    upload_button,
    file_info,
    widgets.HTML("<b>🎵Processed files available:</b>"),
    audio_dropdown,
    play_button,
    status_text,
    delete_ui,
], layout=widgets.Layout(padding='20px', border='solid 1px #ccc', border_radius='10px'))

display(final_ui)

Numero di segmenti: 263, Forma di ogni segmento: (64000,)
Explosion: 0.429
0:00:01.200000


  0%|          | 0/5 [00:00<?, ?it/s]

Rain: 0.458
0:00:02.800000


  0%|          | 0/5 [00:00<?, ?it/s]

Rain on surface: 0.581
0:00:08.100000


  0%|          | 0/5 [00:00<?, ?it/s]

Rain: 0.542
0:00:08.700000


  0%|          | 0/5 [00:00<?, ?it/s]

Wind noise (microphone): 0.417
0:00:21.200000


  0%|          | 0/5 [00:00<?, ?it/s]